# Converting Yelp Dataset json files to CSV

Medium Article - [Convert Yelp Dataset to CSV using Pandas](https://link.medium.com/0k0DEb3Qy1)

[GitHub Repo](https://github.com/gyhou/yelp_dataset)

## Data Source
https://www.yelp.com/dataset/
- Unzip the first .tar file, it will extract a single file
- Add .tar to the end of the file's name to unzip it again
- The second file will extract 6 json files

In [16]:
# Need to upgrade pandas to version 0.25 to use "explode" function
# !py -m venv venv
!venv\Scripts\activate
# !pip install pandas

In [17]:
import pandas as pd
pd.__version__

'2.3.3'

## To match the review texts with business info, we need to merge the two json files

### Load and Clean Business.json file

In [18]:
# Set up your local path
business_json_path = 'business.json'
business = pd.read_json(business_json_path, lines=True)
print(business.shape)
business.head()

(150346, 14)


,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,0,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",None
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,15,1,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Ma...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ..."
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,AZ,85711,32.223236,-110.880452,3.5,22,0,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Department Stores, Shopping, Fashion, Home & G...","{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', ..."
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc...","Brewpubs, Breweries, Food","{'Wednesday': '14:0-22:0', 'Thursday': '16:0-2..."


In [19]:
business.to_csv("yelp_business.csv", index=False) # Save as CSV file without the index column

In [20]:
# There are significant amount of businesses that are not open anymore
business.is_open.value_counts()

is_open
1    119698
0     30648
Name: count, dtype: int64

In [21]:
# Keep only business that are still open
# Drop columns that may not be relavent
df_business = business[business['is_open']==1].drop(['hours','is_open','review_count'], axis=1)
print(df_business.shape)
df_business.head()

(119698, 11)


,business_id,name,address,city,state,postal_code,latitude,longitude,stars,attributes,categories
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Ma..."
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B..."
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc...","Brewpubs, Breweries, Food"
5,CF33F8-E6oudUQ46HnavjQ,Sonic Drive-In,615 S Main St,Ashland City,TN,37015,36.269593,-87.058943,2.0,"{'BusinessParking': 'None', 'BusinessAcceptsCr...","Burgers, Fast Food, Sandwiches, Food, Ice Crea..."
6,n_0UpQx1hsNbnPUSlodU8w,Famous Footwear,"8522 Eager Road, Dierbergs Brentwood Point",Brentwood,MO,63144,38.627695,-90.340465,2.5,"{'BusinessAcceptsCreditCards': 'True', 'Restau...","Sporting Goods, Fashion, Shoe Stores, Shopping..."


### Find relevant categories

One way to split the reviews is by category

In [22]:
# df.explode requires pandas ver 0.25
# Create one row for each series that contain comma-separated items
df_explode = df_business.assign(categories = df_business.categories.str.split(', ')).explode('categories')
df_explode.sample(9)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,attributes,categories
87604,h2-mPUMYJ5pF_gVurDL3lw,Azian Restaurant Sushi & Korean BBQ,15 N Alvernon Way,Tucson,AZ,85711,32.222275,-110.909892,2.5,"{'RestaurantsReservations': 'True', 'OutdoorSe...",Food
145602,USaA0VJQf5EoD7zsIZP1Ag,Hyundai of New Port Richey,3936 US 19,New Port Richey,FL,34652,28.216025,-82.737035,2.5,"{'BusinessAcceptsCreditCards': 'True', 'ByAppo...",Automotive
74561,fYwa1dnZC8LiII2ArgDDoQ,Richie's Hot Wings,2934 Highway 31 W,White House,TN,37188,36.462546,-86.660690,4.0,"{'NoiseLevel': 'u'quiet'', 'BikeParking': 'Tru...",Chicken Wings
3419,2K19T1WL1EqTPs4KYjrkeQ,AFC URGENT CARE SEMINOLE,"11241 Park Blvd, Ste A",Seminole,FL,33772,27.841765,-82.794712,2.0,"{'BusinessAcceptsCreditCards': 'True', 'Accept...",Doctors
99049,4xu5AxIquZThauX74w5hSA,Tullycross Fine Irish Handcrafts,110 South St,Philadelphia,PA,19147,39.940900,-75.144437,5.0,"{'RestaurantsPriceRange2': '2', 'BusinessAccep...",Arts & Crafts
140804,YLR0La5ptvM2b2F9KHS5-g,Best-One of Indy,11578 Allisonville Rd,Fishers,IN,46038,39.955175,-86.041217,4.0,"{'WiFi': 'u'free'', 'BusinessAcceptsCreditCard...",Auto Repair
133182,7JfmABOQj-OKTsf0ay3-cg,Applebee's Grill + Bar,610 West Nicholson Rd,Audubon,NJ,08106,39.893008,-75.087907,2.0,"{'RestaurantsAttire': 'u'casual'', 'GoodForKid...",Bars
27127,KjwrNpjp0wkovwKydQ3tWw,Sushi Stop,882 Jacksonville Rd,Ivyland,PA,18974,40.203297,-75.076035,4.5,"{'BusinessAcceptsCreditCards': 'True', 'Alcoho...",Ramen
99507,B71qd7T1voSiewKHUWD1cg,Summit Acupuncture Philadelphia,"421 N 7th St, Ste 205",Philadelphia,PA,19123,39.958919,-75.150019,5.0,"{'ByAppointmentOnly': 'True', 'AcceptsInsuranc...",Traditional Chinese Medicine


In [23]:
print('Total number of categories:', len(df_explode.categories.value_counts()))
print('Top 10 categories:')
df_explode.categories.value_counts()[:10]

Total number of categories: 1302
Top 10 categories:


categories
Restaurants                  34987
Food                         20419
Shopping                     20186
Home Services                13322
Beauty & Spas                12263
Health & Medical             11046
Local Services               10138
Automotive                    9879
Nightlife                     8379
Event Planning & Services     8173
Name: count, dtype: int64

In [24]:
# Finding categories that contains RV
df_explode[df_explode['categories'].str.contains('RV', case=True, na=False)].categories.value_counts()

categories
RV Repair     103
RV Dealers    103
RV Parks       82
RV Rental      54
Name: count, dtype: int64

In [25]:
# Keep only business with categories that are RV related (including Campgrounds)
business_RV = df_business[df_business['categories'].str.contains(
                         'RV Repair|RV Dealers|RV Rental|RV Parks|Campgrounds', case=False, na=False)]
print(business_RV.shape)
business_RV.head()

(284, 11)


,business_id,name,address,city,state,postal_code,latitude,longitude,stars,attributes,categories
219,25Uww0C0wvF9CZ_3B6vWtA,Enjoy The Mountain,1 Garden St,Santa Barbara,CA,93105,34.414342,-119.687332,4.5,"{'WheelchairAccessible': 'True', 'BikeParking'...","Motorcycle Rental, Tours, Hiking, Mountain Bik..."
225,cSeA6pC9JUlGfz-RNfvnGw,Midwest RV Center,6200 Heimos Industrial Park Dr,St. Louis,MO,63129,38.464156,-90.371669,4.0,"{'BikeParking': 'True', 'RestaurantsPriceRange...","Trailer Dealers, Auto Parts & Supplies, Auto R..."
961,nazEG-Y2LF4nL4bIIz7DKg,Clearwater Travel Resort,2946 Gulf To Bay Blvd,Clearwater,FL,33759,27.962428,-82.714922,3.5,{'WiFi': 'u'free''},"Hotels & Travel, RV Parks, Campgrounds"
1777,nfNOqJTJa-jAHxiyZSyY8g,Woody's RV World,14510 Mark Messier Trail NW,Edmonton,AB,T6V 1H5,53.607767,-113.586814,1.0,None,"Automotive, RV Repair, Trailer Dealers, RV Dea..."
2019,p7RzIdqbVGD-qaqPx2WRtw,Fretz RV,3479 Bethlehem Pike,Souderton,PA,18964,40.318542,-75.303180,3.0,"{'WiFi': 'u'free'', 'ByAppointmentOnly': 'True...","RV Parks, Car Dealers, RV Dealers, RV Repair, ..."


### Loading Massive file as chunks in Pandas

With a huge file like yelp dataset, loading all the data at once will most likely crash your computer.

Luckily, pandas has an option to load large data by segmenting the file into smaller chunks.

In [26]:
# Set up your local path
review_json_path = 'review.json'

In [27]:
import pandas as pd

# Set chunk size (smaller if dataset is smaller)
# 2019 Yelp review.json has more than 6 million reviews(rows)
size = 100000
review = pd.read_json(review_json_path, lines=True,
                      # identifying the data type of each column can reduce memory usage
                      dtype={'review_id':str,'user_id':str,'business_id':str,'stars':int,
                             'date':str,'text':str,'useful':int,'funny':int,'cool':int},
                      chunksize=size)

# review = pd.read_json(review_json_path, lines=True, chunksize=size)

# # Contar número total de linhas processadas
# total = 0
# for chunk in review:
#     total += len(chunk)
#     print(f"Processado {total} linhas até agora...")

# print(f"Total de linhas: {total}")


In [28]:
# # To display dataframe in Jupyter Notebook if not auto displayed using print
# from IPython.display import display

# with open(review_json_path, 'r', encoding='utf-8') as f:
#     review = pd.read_json(f, lines=True, chunksize=size)
#     first_chunk = next(review)
#     print(first_chunk.shape)
#     display(first_chunk)

### Merge Review.json and Business.json file

By merging only the relevant businesses to the review file, the final dataset will only consist of reviews from those businesses

In [29]:
# There are multiple chunks to be read
chunk_list = []
for chunk in review:
    # Drop columns that aren't needed
    chunk = chunk.drop(['review_id','useful','funny','cool'], axis=1)
    # Renaming column name to avoid conflict with business overall star rating
    chunk = chunk.rename(columns={'stars': 'review_stars'})
    # Inner merge with edited business file so only reviews related to the business remain
    chunk_merged = pd.merge(business_RV, chunk, on='business_id', how='inner')
    # Show feedback on progress
    print(f"{chunk_merged.shape[0]} out of {size:,} related reviews")
    chunk_list.append(chunk_merged)

# After trimming down the review file, concatenate all relevant data back to one dataframe
df = pd.concat(chunk_list, ignore_index=True, join='outer', axis=0)
print(df.shape)
df.sample(3)

55 out of 100,000 related reviews
73 out of 100,000 related reviews
70 out of 100,000 related reviews
134 out of 100,000 related reviews
181 out of 100,000 related reviews
215 out of 100,000 related reviews
267 out of 100,000 related reviews
54 out of 100,000 related reviews
31 out of 100,000 related reviews
34 out of 100,000 related reviews
54 out of 100,000 related reviews
82 out of 100,000 related reviews
95 out of 100,000 related reviews
162 out of 100,000 related reviews
105 out of 100,000 related reviews
75 out of 100,000 related reviews
81 out of 100,000 related reviews
113 out of 100,000 related reviews
145 out of 100,000 related reviews
212 out of 100,000 related reviews
227 out of 100,000 related reviews
61 out of 100,000 related reviews
54 out of 100,000 related reviews
63 out of 100,000 related reviews
64 out of 100,000 related reviews
107 out of 100,000 related reviews
118 out of 100,000 related reviews
123 out of 100,000 related reviews
106 out of 100,000 related reviews


,business_id,name,address,city,state,postal_code,latitude,longitude,stars,attributes,categories,user_id,review_stars,text,date
7964,EO0ZCmJv2ldtmI2E0DF59w,La Mesa RV,3255 E Irvington Rd,Tucson,AZ,85714,32.163252,-110.922955,2.5,{'BusinessAcceptsCreditCards': 'True'},"Car Dealers, Auto Parts & Supplies, Automotive...",f9W-s412j8oPiweup9sSmw,1,"We bought two rvs here, La Mesa Tucson. First ...",2019-06-13 21:16:19
4972,SlkS9h_7QaFM2olSCPhBZw,Mount Comfort RV,5935 W 225th N,Greenfield,IN,46140,39.817991,-85.911764,2.5,"{'WiFi': 'u'free'', 'BusinessAcceptsCreditCard...","RV Dealers, Hotels & Travel, Auto Parts & Supp...",KSCLNFNcDiZm4q98LjvBBA,5,I've recommend Mount Comfort RV to a lot of my...,2015-06-23 16:19:52
6138,z4DO84nKcdDNs9cvOlQ5Rw,Lazydays RV of Tampa,6130 Lazy Days Blvd,Seffner,FL,33584,28.006551,-82.308538,2.5,"{'BusinessAcceptsCreditCards': 'True', 'WiFi':...","RV Dealers, RV Parks, Automotive, RV Repair, H...",ljYRQxsI0UY4i1MtkXu0-g,1,If anyone is planning to buy an RV please don'...,2015-10-01 12:01:09


### Convert the new dataframe into a CSV file

By doing so, the smaller dataset can be loaded and shared much more readily

In [ ]:
df.to_csv("yelp_reviews_RV_categories.csv", index=False)